Reproduce Guinness example With OSRM docker image

In [1]:
import geopandas as gpd
import pandas, numpy, pyvrp, sys

In [2]:
sys.path.insert(0, '/home/dylan/projects/gsoc2025/spopt/') # not the published spopt

In [3]:
import spopt
print(spopt.__file__)

/home/dylan/projects/gsoc2025/spopt/spopt/__init__.py


In [4]:
from spopt.route import engine, heuristic, utils

## Reproduce the guinness example

In [5]:
from spopt.route.heuristic import LastMile
from pyvrp import stop

In [6]:
trucks = pandas.DataFrame(
    [['big', 'lng',      2000,    280, .004,  .50, 5],
     ['big', 'electric', 2000,    480, .002,  .50, 5],
     ['med', 'lng',      800, 280*.66, .0001, .63, 10],
     ['med', 'electric', 800, 480*.66, .004,  .50, 10],
     ['smo', 'lng',      400, 280*0.4, .002,  .50, 20],
     ['smo', 'electric', 400, 480*0.4, .0001, .63, 20],
     ],
     columns = [
         'namesize', 'namefuel', 'capacity', 
         'fixed_cost', 'cost_per_meter', 'cost_per_minute', 'n_truck'
         ]
)

In [7]:
dublin_pubs = gpd.read_file('/home/dylan/projects/gsoc2025/spopt/notebooks/gsoc2025/data/dublinpubs.geojson')

In [8]:
dublin_pubs.shape

(551, 8)

In [9]:
gdf = dublin_pubs

In [10]:
depot = gdf.iloc[0,:]
clients = gdf.iloc[1:,:].reset_index(drop=True)
clients = clients.set_index(clients.osmid.astype(str))

In [11]:
_clients = clients[:90]

I am running into rate limits using the osrm docker image. When using the full set of clients (e.g. all 551 pubs), the returned json content from osrm (engine.py, line 163 when trying to parse `content`) is:

```
{'code': 'TooBig', 'message': 'Too many table coordinates'}
```

In [12]:
print('initializing model')
m = LastMile(
    depot_location=(depot.longitude.item(), depot.latitude.item()),
    depot_open=pandas.Timestamp("2030-01-02 07:00:00"),
    depot_close=pandas.Timestamp("2030-01-02 20:00:00"),
    depot_name=depot['name'],
)
print("adding clients")

m.add_clients(
    locations = _clients.geometry, 
    delivery = _clients.demand,
    pickup = _clients.supply,
    time_windows=None,
    service_times=(numpy.log(_clients.demand)**2).astype(int)
)
print("adding trucks")
m.add_trucks_from_frame(
    trucks, 
)

initializing model
adding clients
adding trucks


In [13]:
m.depot_location

(-6.28688, 53.341972)

In [14]:
m.solve(stop=pyvrp.stop.MaxRuntime(300))

PyVRP v0.11.3

Solving an instance with:
    1 depot
    90 clients
    70 vehicles (6 vehicle types)

                  |       Feasible        |      Infeasible
    Iters    Time |   #      Avg     Best |   #      Avg     Best
H     500      9s |  33 10656306  7702446 |  58  7552892  7442864
     1000     16s |  26 11378776  7702446 |  47  7746625  7674608
     1500     23s |  34  7732870  7702446 |  65  6375223  6116902
     2000     30s |  25  7728946  7702446 |  64  7181224  7083976
     2500     37s |  36  7800296  7702446 |  36  5407215  4917004
     3000     44s |  25  7717886  7702446 |  35  7454999  7401870
     3500     51s |  63  7732939  7702446 |  28  6178615  5948820
     4000     57s |  62  7742206  7702446 |  60  5958374  5515747
     4500     64s |  27  7722186  7702446 |  31  7949287  7713186
     5000     71s |  55  7755851  7702446 |  36  8030015  7713790
     5500     78s |  30  7719508  7702446 |  55  5695164  5311965
     6000     84s |  31  7723514  7702446 |  

In [16]:
m.write_result("docker-image")